In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
import scipy.stats as stats

# Parameters

* $\phi$ : transition rate (abstraction of quantum yield)
* $\beta$ : background activity
* $k$ : back-reaction rate coefficient; $k = \frac{\ln{2}}{T_{1/2}}$
* $T_{1/2}$ : back-reaction half-life
* $\epsilon$ : molar absorption coefficient ($M^{-1}cm^{-1}$)
* $l \approx 10 \mu m$ : optical path length

# Intermediates
* $c = [OFF]$ : concentration of molecules in OFF state ($M = mol/L$)
* $A = \epsilon \cdot c \cdot l$ : absorbance (Beer-Lambert law)
* $w = T = 10^{-A}$ : transmittance, i.e. synaptic weight

# Signals
* $x$ : integrand (TN2 - TB1)
* $y$ : output to post-synaptic neurons; ideally $y(t) = \int_0^t x(\tau) d\tau$
* $u = x + \beta$ : output from PFN / CPU4
* $y = wu$

# Dynamics
The concentration changes with two terms:

$$ c' = -kc + \phi u $$

where $-kc$ is the first-order back-reaction and $\phi u$ models the increase in molecules in the OFF state in proportion to the signal (light intensity).

In [ ]:
# Inputs
t0 = 0
tf = 10
T = np.linspace(t0, tf, 100)

def x(t):
    v = stats.norm(3, 1)
    return stats.norm(loc=4, scale=1).pdf(t)

def u(t):
    return x(t) + beta

plt.plot(T, x(T), label="x(t)")
plt.plot(T, u(T), label="u(t)")
plt.legend()

In [ ]:
# Parameters
phi = 1.0
beta = 1.0
T_half = 1.0
k = np.log(2) / T_half
epsilon = 1e2
l = 10e-6 * 1e2

In [ ]:
def dcdt(t, c):
    return -k * c + phi * u(t)

solution = solve_ivp(dcdt, y0 = np.array([0.0]), t_span=(t0, tf), t_eval = T)
c = interp1d(T, solution.y[0])

plt.plot(T, x(T), label="x(t)")
plt.plot(T, u(T), label="u(t)")
plt.plot(T, c(T), label="c(t)")
plt.legend()

In [ ]:
def y(t):
    A = epsilon * c(t) * l
    w = 10 ** -A
    return w * u(t)


plt.plot(T, x(T), label="x(t)")
plt.plot(T, u(T), label="u(t)")
plt.plot(T, c(T), label="c(t)")
plt.plot(T, y(T), label="y(t)")
plt.legend()